In [1]:
import pandas as pd
import numpy as np
import csv
import re
from scipy import stats
import datetime
import functions

# Reading geo-stations data

In [2]:
stations = pd.read_csv("./traffic_flux/stations_geo.csv",sep=";",encoding="latin-1")

# We create a new column with the name of the district
distritos = list(map(lambda x: x/1,range(1,22)))
distritos_name = ['CENTRO','ARGANZUELA','RETIRO','SALAMANCA','CHAMARTIN','TETUAN',
                  'CHAMBERI','FUENCARRAL-ELPARDO','MONCLOA-ARAVACA','LATINA',
                  'CARABANCHEL','USERA','PUENTEDEVALLECAS','MORATALAZ','CIUDADLINEAL',
                  'HORTALEZA','VILLAVERDE','VILLADEVALLECAS','VICÁLVARO',
                  'SANBLAS-CANILLEJAS','BARAJAS']

for distrito,distrito_name in zip(distritos,distritos_name):
    #print(distrito,distrito_name)
    stations.loc[stations['distrito']==distrito,'district'] = distrito_name
    
# filtering the center of madrid
centro = stations[stations['id'].isin([5463,5512,5462])|(stations["district"].isin(['ARGANZUELA','CENTRO','CHAMARTIN','CHAMBERI','MONCLOA-ARAVACA','RETIRO','SALAMANCA','TETUAN','FUENCARRAL-ELPARDO']))]
 
# filtering aravaca with a line:
centro = centro[(centro['district']!='MONCLOA-ARAVACA')|(centro['longitud']>-0.589130*(centro['latitud']-40.472881)-3.762592)]

# filtering fuencarral with a line:
centro = centro[(centro['district']!='FUENCARRAL-ELPARDO')|(centro['longitud']>3.326767*(centro['latitud']-40.480496)-3.714323)]


# taking surroundings of madrid 
colindantes = stations[(stations["district"].isin(['CARABANCHEL','CIUDADLINEAL','LATINA','MORATALAZ','PUENTEDEVALLECAS','USERA']))]

# filtering out everything that is not M30
colindantes = colindantes.loc[(colindantes['nombre'].str.contains("M-30"))|(stations['tipo_elem']=='M30')]
# concatenating back
stations = pd.concat([centro,colindantes],axis=0)

stations['zone'] = np.where((stations['nombre'].str.contains("M-30"))|(stations['tipo_elem']=='M30'),'B','A')

stations

,tipo_elem,distrito,id,cod_cent,nombre,utm_x,utm_y,longitud,latitud,district,zone
0,URB,4.0,3840,01001,Jose Ortega y Gasset E-O - PÂº Castellana-Serrano,441615.343347,4.475768e+06,-3.688323,40.430502,SALAMANCA,A
1,URB,4.0,3841,01002,Jose Ortega y Gasset O-E - Serrano-PÂº Castellana,441705.882340,4.475770e+06,-3.687256,40.430524,SALAMANCA,A
2,URB,1.0,3842,01003,PÂº Recoletos N-S - Almirante-Prim,441319.371258,4.474841e+06,-3.691727,40.422132,CENTRO,A
3,URB,4.0,3843,01004,PÂº Recoletos S-N - Pl. Cibeles- Recoletos,441301.632986,4.474764e+06,-3.691929,40.421433,SALAMANCA,A
4,URB,4.0,3844,01005,(AFOROS) PÂº Castellana S-N - Eduardo Dato - ...,441605.765072,4.476132e+06,-3.688470,40.433782,SALAMANCA,A
...,...,...,...,...,...,...,...,...,...,...,...
4621,M30,10.0,6795,PM41852,PM41852,435158.457286,4.472365e+06,-3.764094,40.399371,LATINA,B
4622,M30,10.0,7012,PM41853,PM41853,435208.574514,4.472347e+06,-3.763501,40.399210,LATINA,B
4623,M30,10.0,10208,PM41861,PM41861,434579.105005,4.471813e+06,-3.770863,40.394352,LATINA,B
4624,M30,10.0,6943,PM41881,PM41881,433504.262880,4.470758e+06,-3.783417,40.384766,LATINA,B


In [3]:
# [OPTIONAL] -- saving for tableau a list with all the stations, and a new column
# stating if the station is within our zone of study or not.

all_stations = pd.read_csv("./traffic_flux/stations_geo.csv",sep=";",encoding="latin-1")

all_stations['tested_zone'] = 'No'
all_stations.loc[stations.index,'tested_zone'] = 'Yes'

all_stations.to_csv('./visualizations/flow_stations.csv')

# Reading 2013-2014 flow data
When joining 2013-2014 data with metadata from stations we use "identif", for 2015-2019 we use ID because the database structure changed.

### 2013

In [4]:
%%time
# Reading air_quality data and concatenating alll the data

flow_2013 = pd.read_csv("./traffic_flux/2013.csv",encoding="latin-1",sep=';')

CPU times: user 1min 4s, sys: 36.9 s, total: 1min 41s
Wall time: 1min 48s


In [6]:
# Inner join with the stations we want to keep
flow_2013 = pd.merge(left = flow_2013, right = stations, how = 'inner', left_on = "identif" , right_on = 'cod_cent')

In [7]:
flow_2013 = flow_2013[['identif','fecha','intensidad','carga','zone']]
flow_2013.columns = ['fix_id','fecha','intensidad','carga','zone']

In [8]:
flow_2013['fecha'] = pd.to_datetime(flow_2013['fecha'])
flow_2013['day'] = flow_2013['fecha'].dt.date

In [9]:
# Averaging over the day
reduced_2013 = flow_2013.groupby(['fix_id','day','zone']).agg({'intensidad':np.mean,'carga':np.mean}).reset_index()

# Averaging over the stations
reduced_2013 = reduced_2013.groupby(['day','zone']).agg({'intensidad':np.mean,'carga':np.mean}).reset_index()
reduced_2013

,day,zone,intensidad,carga
0,2013-01-01,A,322.210156,14.452064
1,2013-01-01,B,948.371939,12.957883
2,2013-01-02,A,445.305361,19.735136
3,2013-01-02,B,1312.361862,17.860123
4,2013-01-03,A,474.192085,20.998438
...,...,...,...,...
725,2013-12-29,B,918.420024,12.673692
726,2013-12-30,A,443.568048,23.651003
727,2013-12-30,B,1283.355318,18.126790
728,2013-12-31,A,334.422604,18.066746


### 2014

In [10]:
#%%time
#flow_2014 = pd.read_csv("./traffic_flux/2014.csv",encoding="latin-1",sep=';',usecols=range(9))

# this full year csv has one whole month missing (november), so we had to use the splitted version (next cell)

In [11]:
%%time 
meses = ['01','02','03','04','05','06','07','08','09','10','11','12']
flow_2014 = pd.DataFrame()
    
for mes in meses:
    separator = functions.get_separator("./traffic_flux/2014/"+str(mes)+"-2014.csv")
    print(mes,separator)
    new_mes = pd.read_csv("./traffic_flux/2014/"+str(mes)+"-2014.csv",sep=separator,encoding="latin-1",on_bad_lines='skip')
    flow_2014 = pd.concat([flow_2014,new_mes],axis=0)

01 ;
02 ;


<timed exec>:7: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.


03 ;
04 ;
05 ;
06 ;
07 ;
08 ;
09 ;
10 ;
11 ;
12 ;
CPU times: user 1min 39s, sys: 45 s, total: 2min 24s
Wall time: 2min 28s


In [12]:
# Converting IDs to numeric
#flow_2014['identif'] =  pd.to_numeric(flow_2014['identif'], errors='coerce')
flow_2014

,identif,fecha,intensidad,ocupacion,carga,tipo,vmed,error,periodo_integracion,idelem,tipo_elem
0,45028,2014-01-14 17:30:00,310,1,36,F,0,N,4,NaN,NaN
1,45414,2014-01-14 17:30:00,27,16,13,F,0,N,3,NaN,NaN
2,53405,2014-01-14 17:30:00,33,23,28,F,0,N,3,NaN,NaN
3,53409,2014-01-14 17:30:00,30,7,19,F,0,N,2,NaN,NaN
4,55041,2014-01-14 17:30:00,520,50,51,F,0,N,4,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
9350697,PM43201,2014-12-23 15:30:00,2608,6,40,M,91,N,15,6809.0,494
9350698,PM43202,2014-12-23 15:30:00,819,4,35,M,107,N,15,6810.0,494
9350699,PM43221,2014-12-23 15:30:00,2812,12,62,M,83,N,15,6933.0,494
9350700,PM43222,2014-12-23 15:30:00,692,8,31,M,73,N,15,7129.0,494


In [13]:
# Inner join with the stations we want to keep
flow_2014 = pd.merge(left = flow_2014, right = stations, how = 'inner', left_on = "identif" , right_on = 'cod_cent')

In [14]:
# Selecting only the columns we need
flow_2014 = flow_2014[['identif','fecha','intensidad','carga','zone']]
flow_2014.columns = ['fix_id','fecha','intensidad','carga','zone']

In [15]:
flow_2014['fecha'] = pd.to_datetime(flow_2014['fecha'])
flow_2014['day'] = flow_2014['fecha'].dt.date

In [16]:
# Averaging over the day
reduced_2014 = flow_2014.groupby(['fix_id','day','zone']).agg({'intensidad':np.mean,'carga':np.mean}).reset_index()

# Averaging over the stations
reduced_2014 = reduced_2014.groupby(['day','zone']).agg({'intensidad':np.mean,'carga':np.mean}).reset_index()
reduced_2014

,day,zone,intensidad,carga
0,2014-01-01,A,283.427006,14.818900
1,2014-01-01,B,789.538789,12.755389
2,2014-01-02,A,414.334112,22.033154
3,2014-01-02,B,1173.802918,17.836658
4,2014-01-03,A,447.462606,23.848786
...,...,...,...,...
725,2014-12-29,B,1290.499570,17.696652
726,2014-12-30,A,1258.234069,25.674020
727,2014-12-30,B,1482.727627,17.829784
728,2014-12-31,A,971.183211,19.757966


# Reading 2015-2019
When joining 2013-2014 data with metadata from stations we use "identif", for 2015-2019 we use ID because the database structure changed.

In [17]:
import functions

In [18]:
%%time 
"""# Reading and concatenating data for all the years. This takes a lot time (20min) and since we have the result
# already saved in a csv we'll just read them in next cell
anios = range(2015,2020)
meses = ['01','02','03','04','05','06','07','08','09','10','11','12']
tablas = []
for i,anio in enumerate(anios):
    tablas.append(pd.DataFrame())
    
    for mes in meses:
        separator = functions.get_separator("./traffic_flux/"+str(anio)+"/"+str(mes)+"-"+str(anio)+".csv")
        print(anio,mes,separator)
        new_mes = pd.read_csv("./traffic_flux/"+str(anio)+"/"+str(mes)+"-"+str(anio)+".csv",sep=separator,encoding="latin-1",on_bad_lines='skip')
        tablas[i] = pd.concat([tablas[i],new_mes],axis=0)
        

# Unifying the id name column
for i in range(3,5):
    lista  = tablas[i].columns[1:]
    lista = lista.insert(0,'idelem')
    tablas[i].columns = lista

for tabla in tablas:
    if tabla['idelem'].isna().sum()>0:
        tabla['fix_id'] = np.where(~tabla['idelem'].isna(),tabla['idelem'],tabla['id'])
    else:
        tabla['fix_id'] = tabla['idelem']
        
for i,tabla in enumerate(tablas):
    tablas[i] = pd.merge(left = tablas[i], right = stations, how = 'inner', left_on = "fix_id" , right_on = 'id')
    
for i,tabla in enumerate(tablas):
    tablas[i] = tabla[['fix_id','fecha','intensidad','carga','zone']]"""

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 12.2 µs


'# Reading and concatenating data for all the years. This takes a lot time and since we have the result already saved\n# in a csv we\'ll just read them in next cell\nanios = range(2015,2020)\nmeses = [\'01\',\'02\',\'03\',\'04\',\'05\',\'06\',\'07\',\'08\',\'09\',\'10\',\'11\',\'12\']\ntablas = []\nfor i,anio in enumerate(anios):\n    tablas.append(pd.DataFrame())\n    \n    for mes in meses:\n        separator = functions.get_separator("./traffic_flux/"+str(anio)+"/"+str(mes)+"-"+str(anio)+".csv")\n        print(anio,mes,separator)\n        new_mes = pd.read_csv("./traffic_flux/"+str(anio)+"/"+str(mes)+"-"+str(anio)+".csv",sep=separator,encoding="latin-1",on_bad_lines=\'skip\')\n        tablas[i] = pd.concat([tablas[i],new_mes],axis=0)\n        \n\n# Unifying the id name column\nfor i in range(3,5):\n    lista  = tablas[i].columns[1:]\n    lista = lista.insert(0,\'idelem\')\n    tablas[i].columns = lista\n\nfor tabla in tablas:\n    if tabla[\'idelem\'].isna().sum()>0:\n        tabla[

In [19]:
%%time
## security backup
#for i,tabla in enumerate(tablas):
#    tabla.to_csv('tabla'+str(i)+'.csv',index=False)

# Reading security  backups
tablas = []
for i in range(5):
    tablas.append(pd.read_csv("tabla"+str(i)+".csv"))

CPU times: user 2min 12s, sys: 29 s, total: 2min 41s
Wall time: 2min 43s


In [20]:
%%time
# Grouping by day, and then by station
reduced_tablas = []
for i,tabla in enumerate(tablas):
    tablas[i]['fecha'] = pd.to_datetime(tablas[i]['fecha'])
    tablas[i]['day'] = tablas[i]['fecha'].dt.date
    tablita = tablas[i].groupby(['fix_id','day','zone']).agg({'intensidad':np.mean,'carga':np.mean}).reset_index()    
    reduced_tablas.append(tablita.groupby(['day','zone']).agg({'intensidad':np.mean,'carga':np.mean}).reset_index())

CPU times: user 3min 8s, sys: 32 s, total: 3min 40s
Wall time: 3min 42s


In [21]:
%%time
# concatenating 2015-2019 together
all_flow = pd.DataFrame()
for tabla in reduced_tablas:
    all_flow = pd.concat([all_flow,tabla],axis=0)
all_flow

CPU times: user 4.85 ms, sys: 8.64 ms, total: 13.5 ms
Wall time: 13.3 ms


,day,zone,intensidad,carga
0,2015-01-01,A,815.194240,17.017157
1,2015-01-01,B,952.059537,11.640500
2,2015-01-02,A,941.155797,23.209692
3,2015-01-02,B,1247.052799,16.191939
4,2015-01-03,A,923.155025,19.348039
...,...,...,...,...
725,2019-12-29,B,898.874443,19.148592
726,2019-12-30,A,387.040593,19.328343
727,2019-12-30,B,1207.982112,25.728488
728,2019-12-31,A,290.240901,14.712693


# Concatenating 2013-14, and then with the rest

In [22]:
flow1314 = pd.concat([reduced_2013,reduced_2014],axis=0)

In [23]:
full_flow = pd.concat([flow1314,all_flow],axis=0).reset_index(drop=True)

In [24]:
full_flow['day'] = pd.to_datetime(full_flow['day'])

In [25]:
# We now create our time_range column
full_flow_trange = functions.aggregate_time2(full_flow,'day')

In [26]:
# and then group by time_range
full_flow_2 = full_flow_trange.groupby(['time_range','zone']).agg({'intensidad':np.mean,'carga':np.mean}).reset_index()
full_flow_2

,time_range,zone,intensidad,carga
0,2013-01-01,A,442.853860,19.480309
1,2013-01-01,B,1273.953168,19.035396
2,2013-01-16,A,471.284904,20.490961
3,2013-01-16,B,1398.558473,21.365628
4,2013-01-31,A,484.401443,9.943076
...,...,...,...,...
337,2019-11-26,B,1219.741258,25.986988
338,2019-12-11,A,430.633538,21.554419
339,2019-12-11,B,1276.067310,27.106701
340,2019-12-26,A,347.732603,17.453053


In [27]:
# We want to change the format of the table, we'll create a new one
columns = ['time_range','I_A','I_B','C_A','C_B']

time_ranges = pd.to_datetime(full_flow_2.time_range.unique())
# Defining a new dataframe.
full_flow_T = pd.DataFrame([[0]*len(columns)],columns=columns)

# Iterating for each time range
for i,trange in enumerate(time_ranges):
    full_flow_T.loc[i,'time_range'] = trange # writing the time range
    full_flow_T.loc[i,'I_A'] = full_flow_2.loc[(full_flow_2['time_range']==trange)&(full_flow_2['zone']=='A'),'intensidad'].values[0]
    
    full_flow_T.loc[i,'I_B'] = full_flow_2.loc[(full_flow_2['time_range']==trange)&(full_flow_2['zone']=='B'),'intensidad'].values[0]
    full_flow_T.loc[i,'C_A'] = full_flow_2.loc[(full_flow_2['time_range']==trange)&(full_flow_2['zone']=='A'),'carga'].values[0]
    full_flow_T.loc[i,'C_B'] = full_flow_2.loc[(full_flow_2['time_range']==trange)&(full_flow_2['zone']=='B'),'carga'].values[0]

In [28]:
full_flow_T['time_range']=pd.to_datetime(full_flow_T['time_range'])
# Snake_case before saving the data
full_flow_T.columns = list(map(lambda x: x.lower(), full_flow_T.columns))
full_flow_T.columns = full_flow_T.columns.str.replace(' ','_')

In [29]:
full_flow_T

,time_range,i_a,i_b,c_a,c_b
0,2013-01-01,442.853860,1273.953168,19.480309,19.035396
1,2013-01-16,471.284904,1398.558473,20.490961,21.365628
2,2013-01-31,484.401443,1426.810268,9.943076,21.982869
3,2013-02-15,450.632878,1363.748618,-46.937710,14.447603
4,2013-03-02,460.375634,1395.568157,7.510698,20.785309
...,...,...,...,...,...
166,2019-10-27,412.680069,1246.107304,20.669958,26.423670
167,2019-11-11,437.872429,1288.917360,21.941238,27.440334
168,2019-11-26,409.747544,1219.741258,20.623928,25.986988
169,2019-12-11,430.633538,1276.067310,21.554419,27.106701


In [30]:
full_flow_T.to_csv('./traffic_flux/clean_flow.csv',index=False)